# Query Using CosmosDB SQL

If CosmosDB were used during the ETL process the following syntax would be available to us. The sql query below would return the table following:

```SQL
SELECT fdic_certificate_number as fdic_id, 
        financial_institution_name as institution,
        reporting_period_end_date as datestring, 
        financials.RCFD2170 as assets_total
FROM ffeic_reports

```

| fdic_id | institution | datestring | assets_total |
|---------|-------------|------------|--------------|
| 3510    | Bank of Am. | 2017-01-01 | 170098.9     |
| 3510    | Bank of Am. | 2017-03-01 | 1699098.9    |

Using the first query as a sub query in the next step would yield the pivoted result.

```SQL
SELECT datestring,
    sum(case when fdic_id = 3510 then assets_total else 0 as aa) as bank_of_america,
    sum(case when fdic_id = 3511 then assets_total else 0 as bb) as wells_fargo,
    sum(case when fdic_id = 628 then assets_total else 0 as cc) as jp_morgan,
    sum(case when fdic_id = 7213 then assets_total else 0 as dd) as citi,
    sum(case when fdic_id = 32188 then assets_total else 0 as ee) as usaa,
FROM (
    SELECT fdic_certificate_number as fdic_id, 
        financial_institution_name as institution,
        reporting_period_end_date as datestring, 
        financials.RCFD2170 as assets_total
    FROM ffeic_reports ) AA
GROUP BY datestring
```

| datestring | bank_of_america | wells_fargo | jp_morgan |   citi  | usaa   |
|------------|-----------------|-------------|-----------|---------|--------|
| 2017-01-01 |  12345.6        |   12345.6   |  12345.6  | 12345.6 | 12345.6|
| 2017-03-01 |  12345.6        |   12345.6   |  12345.6  | 12345.6 | 12345.6|
| 2017-06-01 |  12345.6        |   12345.6   |  12345.6  | 12345.6 | 12345.6|

Due to the structure of the records we can filter on date and fdic identifier as well. If these were larger tables the indicies defined on these keys would facilitate optimized query speeds.

```SQL
SELECT fdic_certificate_number as fdic_id, 
        financial_institution_name as institution,
        reporting_period_end_date as datestring, 
        financials.RCFD2170 as assets_total
FROM ffeic_reports
WHERE reporting_period_end_date >= '2017-01-01' and 
    fdic_certificate_number in (3510, 3511, 628, 7213, 32188)
```

Placing this query into the larger query for pivoting above would yield an optimal filtered query in the pivoted format. 

```SQL
SELECT datestring,
    sum(case when fdic_id = 3510 then assets_total else 0 as aa) as bank_of_america,
    sum(case when fdic_id = 3511 then assets_total else 0 as bb) as wells_fargo,
    sum(case when fdic_id = 628 then assets_total else 0 as cc) as jp_morgan,
    sum(case when fdic_id = 7213 then assets_total else 0 as dd) as citi,
    sum(case when fdic_id = 32188 then assets_total else 0 as ee) as usaa,
FROM (
    SELECT fdic_certificate_number as fdic_id, 
        financial_institution_name as institution,
        reporting_period_end_date as datestring, 
        financials.RCFD2170 as assets_total
    FROM ffeic_reports
    WHERE reporting_period_end_date >= '2017-01-01' and 
        fdic_certificate_number in (3510, XXXX, XXXX, XXXX)) AA
GROUP BY datestring
```